<p>Измалкова Дарья БКЛ182</p>

<h1>Домашнее задание №2 часть 1</h1>

In [1]:
from pymystem3 import Mystem
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from pymorphy2 import MorphAnalyzer
from collections import Counter
import json
import re
import os


morph = MorphAnalyzer()
m = Mystem()

In [2]:
with open('igrok.txt', encoding='utf-8') as f:
    text = f.read()

<h3>Токенизация с помощью nltk</h3>

In [3]:
%%time 
(word_tokenize(text))

words = [w.lower() for w in word_tokenize(text) if w.isalpha()]

Wall time: 2.03 s


<h3>Биграммы  и Триграммы</h3>

In [4]:
def find_ngrams(words, number):
    ngram_list = list(ngrams(words, number))
    ngrams_rank = Counter(ngram_list)
    top_20 = ngrams_rank.most_common(20)
    for ngram in top_20:
        n_words = ' '.join(ngram[0])
        frequency = ngram[1]
        print(n_words, frequency)


find_ngrams(words, 2)

что я 116
mademoiselle blanche 96
и не 89
может быть 75
я не 73
мистер астлей 69
всё это 55
и в 54
потому что 51
на меня 47
и я 47
что вы 47
у меня 45
что она 44
тотчас же 42
ничего не 42
не знаю 41
и даже 41
ко мне 40
алексей иванович 40


In [5]:
find_ngrams(words, 3)

по крайней мере 27
в самом деле 18
до сих пор 18
пятьдесят тысяч франков 15
к тому же 12
даже и не 12
и в самом 11
может быть и 10
о том что 10
на этот раз 10
что я не 10
я ничего не 10
во всяком случае 9
и mademoiselle blanche 9
что же касается 9
же касается до 9
и уж конечно 8
я так и 8
mademoiselle blanche и 8
а между тем 8


<h3>Парсинг с помощью pymorphy</h3>

In [7]:
%%time
analist = []

for word in words:
    ana = morph.parse(word)
    first = ana[0]
    word = first.word
    tag = first.tag
    lemma = first.normal_form
    prob = first.score
    dic = {'text' : word, 'tag' : [tag], 'lemma' : lemma, 'prob' : prob}
    analist.append(dic)

Wall time: 21.4 s


<h3>Доля частей речи</h3>

In [8]:
word_count = len(analist)
poslist = []
for parse in analist:
    poslist.append(parse['tag'][0].POS)
posdict = Counter(poslist)
for pos, count in posdict.items():
    print('Доля', pos, str(round(count/word_count * 100, 2))+'%')

Доля None 2.69%
Доля NOUN 18.72%
Доля PREP 9.6%
Доля ADJF 9.23%
Доля ADVB 7.61%
Доля NPRO 11.49%
Доля VERB 13.71%
Доля NUMR 1.05%
Доля CONJ 12.47%
Доля PRCL 6.89%
Доля INFN 2.8%
Доля INTJ 0.23%
Доля GRND 0.8%
Доля PRED 0.51%
Доля COMP 0.32%
Доля ADJS 1.03%
Доля PRTS 0.28%
Доля PRTF 0.58%


<h3>Топ-20 глаголов и наречий</h3>

In [9]:
def top_pos(analist,tag):
    lemmalist = []
    for parse in analist:
        if tag in parse['tag'][0]:
            lemmalist.append(parse['lemma'])
    lemmadict = Counter(lemmalist)
    top_list = []
    for key, value in lemmadict.most_common(20):
        top_list.append(key)
    return top_list
    
    
print('Топ-20 глаголов:',top_pos(analist,'VERB'))
print('Топ-20 наречий:',top_pos(analist,'ADVB'))

Топ-20 глаголов: ['быть', 'мочь', 'знать', 'сказать', 'хотеть', 'говорить', 'стать', 'думать', 'видеть', 'выйти', 'проиграть', 'заметить', 'дать', 'взять', 'любить', 'понимать', 'смотреть', 'отвечать', 'начать', 'выиграть']
Топ-20 наречий: ['теперь', 'вдруг', 'уж', 'уже', 'очень', 'опять', 'здесь', 'тоже', 'тут', 'потому', 'тогда', 'почти', 'тотчас', 'наконец', 'более', 'совсем', 'потом', 'где', 'сейчас', 'действительно']


<h3>Чтобы можно было отдельно вытаскивать теги, не используя встроенные функции OpencorporaTag</h3>

In [10]:
for parse in analist:
    new_thing = {}
    if parse['tag'][0].POS:
        new_thing.update({'POS':parse['tag'][0].POS})
    if parse['tag'][0].animacy:
        new_thing.update({'animacy':parse['tag'][0].animacy})
    if parse['tag'][0].aspect:
        new_thing.update({'aspect':parse['tag'][0].aspect})
    if parse['tag'][0].case:
        new_thing.update({'case':parse['tag'][0].case})
    if parse['tag'][0].gender:
        new_thing.update({'gender':parse['tag'][0].gender})
    if parse['tag'][0].involvement:
        new_thing.update({'involvement':parse['tag'][0].involvement})
    if parse['tag'][0].mood:
        new_thing.update({'mood':parse['tag'][0].mood})
    if parse['tag'][0].number:
        new_thing.update({'number':parse['tag'][0].number})
    if parse['tag'][0].person:
        new_thing.update({'person':parse['tag'][0].person})
    if parse['tag'][0].tense:
        new_thing.update({'tense':parse['tag'][0].tense})
    if parse['tag'][0].transitivity:
        new_thing.update({'transitivity':parse['tag'][0].transitivity})
    if parse['tag'][0].voice:
        new_thing.update({'voice':parse['tag'][0].voice})
    parse['tag'].append(new_thing)

print(analist[:5])

[{'text': 'annotation', 'tag': [OpencorporaTag('LATN'), {}], 'lemma': 'annotation', 'prob': 1.0}, {'text': 'роман', 'tag': [OpencorporaTag('NOUN,anim,masc,Name sing,nomn'), {'POS': 'NOUN', 'animacy': 'anim', 'case': 'nomn', 'gender': 'masc', 'number': 'sing'}], 'lemma': 'роман', 'prob': 0.08333333333333333}, {'text': 'из', 'tag': [OpencorporaTag('PREP'), {'POS': 'PREP'}], 'lemma': 'из', 'prob': 0.999661}, {'text': 'записок', 'tag': [OpencorporaTag('NOUN,inan,femn plur,gent'), {'POS': 'NOUN', 'animacy': 'inan', 'case': 'gent', 'gender': 'femn', 'number': 'plur'}], 'lemma': 'записка', 'prob': 1.0}, {'text': 'молодого', 'tag': [OpencorporaTag('ADJF,Qual neut,sing,gent'), {'POS': 'ADJF', 'case': 'gent', 'gender': 'neut', 'number': 'sing'}], 'lemma': 'молодой', 'prob': 0.625}]


<h3>Формат json не признает OpencorporaTag, поэтому переделала тег в строку и убрала "OpencorporaTag", оставив только непосредственно теги</h3>

In [11]:
with open('igrok_parse.json', 'w', encoding='utf-8') as f:

    for parse in analist:
        parse['tag'][0] = str(parse['tag'][0])
        parse['tag'][0] = re.sub(r'OpencorporaTag()', '', parse['tag'][0])

        json.dump(parse, f)
        f.write('\n')

<p>Проверка, что файл нормально записался</p>

In [12]:
ana_list = []
with open('igrok_parse.json', encoding='utf-8') as f:
    text = f.read()
    lines = text.splitlines()

for line in lines:
    data = json.loads(line)
    ana_list.append(data)

print(ana_list[:5])

[{'text': 'annotation', 'tag': ['LATN', {}], 'lemma': 'annotation', 'prob': 1.0}, {'text': 'роман', 'tag': ['NOUN,anim,masc,Name sing,nomn', {'POS': 'NOUN', 'animacy': 'anim', 'case': 'nomn', 'gender': 'masc', 'number': 'sing'}], 'lemma': 'роман', 'prob': 0.08333333333333333}, {'text': 'из', 'tag': ['PREP', {'POS': 'PREP'}], 'lemma': 'из', 'prob': 0.999661}, {'text': 'записок', 'tag': ['NOUN,inan,femn plur,gent', {'POS': 'NOUN', 'animacy': 'inan', 'case': 'gent', 'gender': 'femn', 'number': 'plur'}], 'lemma': 'записка', 'prob': 1.0}, {'text': 'молодого', 'tag': ['ADJF,Qual neut,sing,gent', {'POS': 'ADJF', 'case': 'gent', 'gender': 'neut', 'number': 'sing'}], 'lemma': 'молодой', 'prob': 0.625}]


In [13]:
%%time
import os

inp = "igrok.txt"
outp = "igrok.json"
mystem_path = os.path.join('/Users/User', 'mystem.exe')

# os.path.abspath - находит абсолютный путь
# os.path.join - объединяет части пути до файла
input_filename = os.path.abspath(inp)
output_filename = os.path.abspath(outp)
os.system(f"{mystem_path} {'-gin'} {input_filename} {output_filename}")

Wall time: 1.91 s


0